In [1]:
import os
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
IMAGE_SHAPE = (148, 148)
batch_size = 32
model_path ="https://tfhub.dev/google/bit/m-r101x3/ilsvrc2012_classification/1"

In [4]:
classes = pd.read_csv("archive/Meta.csv")["ClassId"].unique()
test_labels = pd.read_csv

In [5]:
tfhub_cache_dir = os.path.join(os.getcwd(),'tfhub_cache')
os.makedirs(tfhub_cache_dir, exist_ok = True)
os.environ['TFHUB_CACHE_DIR'] = tfhub_cache_dir

### Dataset Config

In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./archive/train",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMAGE_SHAPE,
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./archive/test",
  labels=None,
  label_mode=None,
  seed=123,
  image_size=IMAGE_SHAPE,
  batch_size=batch_size)


Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 12630 files belonging to 1 classes.


In [8]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Model Building and Traning

In [6]:
# get the classifier without head
classifier = hub.KerasLayer(
    model_path, input_shape=IMAGE_SHAPE+(3,), trainable=False)

In [9]:
classifier.trainable = False  # Freeze the outer model

In [10]:
num_classes = len(classes)

model = tf.keras.Sequential([
  classifier,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              387934888 
                                                                 
 dense (Dense)               (None, 43)                43043     
                                                                 
Total params: 387,977,931
Trainable params: 43,043
Non-trainable params: 387,934,888
_________________________________________________________________


In [11]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [12]:
history = model.fit(train_ds, epochs=10)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

failed to allocate memory
	 [[{{node unit14/c/standardized_conv2d/sub}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_157957]

In [ ]:
accuracy  = history["accuracy"]
val_accuracy  = history["val_accuracy"]
epochs = len(accuracy)


plt.plot(epochs, accuracy, 'y', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
loss  = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

### Check Predictions

In [ ]:
## TODO get correct image comparision from Meta folder
predicted_batch = model.predict(test_ds)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = classes[predicted_id]

In [ ]:
m = tf.keras.metrics.Accuracy()
m.update_state(predicted_batch, [[0], [2], [3], [4]])
m.result().numpy()

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5, n+1)
  plt.imshow(test_ds[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

### Save Model

In [ ]:
export_path = os.path.join(os.getcwd(),'saved_models','model')
model.save(export_path)